# Overview
The data has been split into two groups:
- training set (train.csv)
- test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

# Загрузка модулей и датасета

In [834]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, roc_auc_score, recall_score, precision_score

import warnings
warnings.filterwarnings('ignore')

In [835]:
RANDOM_SEED = 17
titanic = pd.read_csv('train.csv')
targets_train = titanic.Survived
data_train = titanic.drop(columns=['Survived', 'Ticket', 'Fare', 'Cabin'])

titanic = pd.read_csv('test.csv')
data_test = titanic.drop(columns=['Ticket', 'Fare', 'Cabin'])


In [836]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Cabin        891 non-null    object 
 8   Embarked     891 non-null    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 62.8+ KB


Видим, что поля Age, Cabin и Embarked заполнены не у всех. Заполним возраст медианным значением для каждого пола, Cabin и Embarked заполним `U`

In [837]:
median_age_male = data_train[data_train['Sex'] == 'male']['Age'].median()
median_age_female = data_train[data_train['Sex'] == 'female']['Age'].median()

In [838]:
data_train['Age'][data_train['Sex'] == 'male'] = data_train['Age'].fillna(median_age_male)
data_train['Age'][data_train['Sex'] == 'female'] = data_train['Age'].fillna(median_age_female)
data_test['Age'][data_test['Sex'] == 'male'] = data_test['Age'].fillna(median_age_male)
data_test['Age'][data_test['Sex'] == 'female'] = data_test['Age'].fillna(median_age_female)

In [839]:
def fillna(data):
    # data['Cabin'] = data['Cabin'].fillna('U')
    data['Embarked'] = data['Embarked'].fillna('U')
fillna(data_train)
fillna(data_test)

Проанализируем в составе имени Mr., Mrs. и т.д, оставив только их

In [840]:
def naming(data):
    for i in range(data.Name.count()):
        s = data['Name'][i]
        new_Name = s[s.find(', ')+2:s.find('. ')+1]
        data['Name'][i] = new_Name
naming(data_train)
naming(data_test)

In [841]:

data_train = data_train.replace(['male', 'female'], [0,1])
data_train = data_train.replace(['U', 'S', 'C', 'Q'], [0,1,2,3])
data_train = data_train.replace(['Mr.', 'Miss.', 'Mrs.', 'Dr.', 'Master.'], [0,1,2,3,4])

for i in range(data_train.Name.count()):
    if data_train['Name'][i] not in [0,1,2,3,4]:
        data_train['Name'][i] = 5 
data_train['Name'] = data_train['Name'].astype(int)
    


In [842]:

data_test = data_test.replace(['male', 'female'], [0,1])
data_test = data_test.replace(['U', 'S', 'C', 'Q'], [0,1,2,3])
data_test = data_test.replace(['Mr.', 'Miss.', 'Mrs.', 'Dr.', 'Master.'], [0,1,2,3,4])

for i in range(data_test.Name.count()):
    if data_test['Name'][i] not in [0,1,2,3,4]:
        data_test['Name'][i] = 5 
data_test['Name'] = data_test['Name'].astype(int)
    


In [843]:
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,892,3,0,0,34.5,0,0,3
1,893,3,2,1,47.0,1,0,1
2,894,2,0,0,62.0,0,0,3
3,895,3,0,0,27.0,0,0,1
4,896,3,2,1,22.0,1,1,1
...,...,...,...,...,...,...,...,...
413,1305,3,0,0,29.0,0,0,1
414,1306,1,5,1,39.0,0,0,2
415,1307,3,0,0,38.5,0,0,1
416,1308,3,0,0,29.0,0,0,1


In [844]:
data_target = targets_train
data_features = data_train

X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target,
    random_state=RANDOM_SEED,
    test_size=0.25
)

In [845]:
knn = KNeighborsClassifier(n_neighbors=15)
lr = LogisticRegression(random_state=RANDOM_SEED)
dtc = DecisionTreeClassifier(max_leaf_nodes=6, random_state=RANDOM_SEED)
svm = SVC(random_state=RANDOM_SEED)

knn_model = knn.fit(X_train, y_train)
lr_model = lr.fit(X_train, y_train)
dtc_model = dtc.fit(X_train, y_train)
svm_model = svm.fit(X_train, y_train)

In [846]:
knn_predict = knn.predict(X_test)
lr_predict = lr.predict(X_test)
dtc_predict = dtc.predict(X_test)
svm_predict = svm.predict(X_test)


In [847]:
recall_score(knn_predict, y_test)

0.38461538461538464

In [848]:
recall_score(lr_predict, y_test)

0.7951807228915663

In [849]:
recall_score(dtc_predict, y_test)

0.84375

In [850]:
recall_score(svm_predict, y_test)

0.0

In [851]:
precision_score(dtc_predict, y_test)

0.5869565217391305

In [853]:
predict = lr.predict(data_test)

In [854]:
predict

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [855]:
answerId = data_test['PassengerId']
answerPr = pd.DataFrame(predict)

In [856]:
answer = pd.concat([answerId, answerPr], axis=1)
answer.columns = ['PassengerId','Survived']

In [857]:
answer

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [858]:
answer.to_csv('answer.csv', index=False)